In [1]:
import numpy as np
import os

In [2]:
# Load DataSet
X_seen=np.load('X_seen.npy',encoding='bytes',allow_pickle=True) 	# (40 x N_i x D): 40 feature matrices. X_seen[i] is the N_i x D feature matrix of seen class i

Xtest=np.load('Xtest.npy')	# (6180, 4096): feature matrix of the test data.

Ytest=np.load('Ytest.npy',)	 # (6180, 1): ground truth labels of the test data

class_attributes_seen=np.load('class_attributes_seen.npy')	# (40, 85): 40x85 matrix with each row being the 85-dimensional class attribute vector of a seen class.

class_attributes_unseen=np.load('class_attributes_unseen.npy')	# (10, 85): 10x85 matrix with each row being the 85-dimensional class attribute vector of an  unseen class.

### Assuming we have a list class_labels that contains class labels for each training input point, where class_labels[i, j] represents the class label for the jth training input point in class i.


In [3]:
# Calculate the sum of the feature vectors for each class after iterating over the training data.
# Calculate Mean of each class
X_class_mean = []

for i in range(40):
    class_mean = np.mean(X_seen[i], axis=0)
    X_class_mean.append(class_mean)

Calculating the weight matrix $ W = \left(A_s^T A_s + \lambda I\right)^{-1} A_s^T M_s $

In [4]:

# Calculating the weight matrix(W) is 85 × 4096 matrix of weights
lambda_ = 10
A_transpose_A = np.dot(class_attributes_seen.T, class_attributes_seen)
I = np.identity(class_attributes_seen.shape[1])
lambda_identity_matrix = lambda_ * I
temp = A_transpose_A + lambda_identity_matrix
inv_temp_matrix = np.linalg.inv(temp)
temp_2 = np.dot(inv_temp_matrix, class_attributes_seen.T)
weight_matrix = np.dot(temp_2, X_class_mean)


## Calculating the the vectorvalued regression output of unseen classes $\mu_c = Wa$

In [5]:
# Calculating the vectorvalued regression output
unseen_class_prototypes = np.zeros((10,4096))

for i in range(10):
    unseen_class_prototypes[i] = (np.dot(weight_matrix.T, class_attributes_unseen[i])).T

In [6]:
# Predicting labels
predicted_labels = []
for i in range(len(Xtest)):
    distances = []
    for j in range(10):
        distance = np.linalg.norm(Xtest[i]-unseen_class_prototypes[j])
        distances.append(distance)
    predict_value=np.argmin(distances)+1
    predicted_labels.append(predict_value)

In [7]:
# Predicting Accuracy
correct_prediction = 0
total_prediction = len(Ytest)
for i in range(total_prediction):
    if(predicted_labels[i]==Ytest[i]):
        correct_prediction+=1

accuracy = correct_prediction/total_prediction*100
print("Accuracy : {}".format(accuracy))

Accuracy : 73.28478964401295


## Now we will run for different lamdas = [0.01, 0.1, 1, 10, 20, 50, 100]

In [8]:
# Running for different lambda
lambdas = [0.01, 0.1, 1, 10, 20, 50, 100]
for lambda_ in lambdas:
    A_transpose_A = np.dot(class_attributes_seen.T, class_attributes_seen)
    I = np.identity(class_attributes_seen.shape[1])
    lambda_identity_matrix = lambda_ * I
    temp = A_transpose_A + lambda_identity_matrix
    inv_temp_matrix = np.linalg.inv(temp)
    temp_2 = np.dot(inv_temp_matrix, class_attributes_seen.T)
    weight_matrix = np.dot(temp_2, X_class_mean)

    # Calculating the vectorvalued regression output
    unseen_class_prototypes = np.zeros((10,4096))

    for i in range(10):
        unseen_class_prototypes[i] = (np.dot(weight_matrix.T, class_attributes_unseen[i])).T
    predicted_labels = []
    for i in range(len(Xtest)):
        distances = []
        for j in range(10):
            distance = np.linalg.norm(Xtest[i]-unseen_class_prototypes[j])
            distances.append(distance)
        predict_value=np.argmin(distances)+1
        predicted_labels.append(predict_value)
    
    # Predicting Accuracy
    correct_prediction = 0
    total_prediction = len(Ytest)
    for i in range(total_prediction):
        if(predicted_labels[i]==Ytest[i]):
            correct_prediction+=1

    accuracy = correct_prediction/total_prediction*100
    print("Accuracy with {}: {}".format(lambda_,accuracy))
    
    

Accuracy with 0.01: 58.090614886731395
Accuracy with 0.1: 59.54692556634305
Accuracy with 1: 67.39482200647248
Accuracy with 10: 73.28478964401295
Accuracy with 20: 71.68284789644012
Accuracy with 50: 65.08090614886731
Accuracy with 100: 56.47249190938511
